In [1]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 데이터셋 클래스 정의
class AutonomousDrivingDataset(Dataset):
    def __init__(self, root_dir, original_transform=None, augment_transform=None):
        self.root_dir = root_dir
        self.original_transform = original_transform
        self.augment_transform = augment_transform
        self.images_dir = os.path.join(root_dir, 'images')
        self.labels_dir = os.path.join(self.root_dir, 'labels')
        self.image_files = sorted(os.listdir(self.images_dir))

    def __len__(self):
        return len(self.image_files) * 2  # 원본 + 증강 데이터

    def __getitem__(self, idx):
        real_idx = idx // 2
        is_augmented = idx % 2 == 1

        img_name = self.image_files[real_idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.jpg', '.json'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        annotations = sorted(label_data['Annotation'], key=lambda x: label_data['Annotation'].index(x), reverse=True)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        class_mapping = {
            'road': 1, 'sidewalk': 2, 'road roughness': 3, 'road boundaries': 4, 'crosswalks': 5,
            'lane': 6, 'road color guide': 7, 'road marking': 8, 'parking': 9, 'traffic sign': 10,
            'traffic light': 11, 'pole/structural object': 12, 'building': 13, 'tunnel': 14,
            'bridge': 15, 'pedestrian': 16, 'vehicle': 17, 'bicycle': 18, 'motorcycle': 19,
            'personal mobility': 20, 'dynamic': 21, 'vegetation': 22, 'sky': 23, 'static': 24
        }

        for annotation in annotations:
            points = np.array(annotation['data'][0]).reshape((-1, 1, 2)).astype(np.int32)
            class_name = annotation['class_name']
            class_id = class_mapping.get(class_name, 0)
            cv2.fillPoly(mask, [points], class_id)

        if is_augmented and self.augment_transform:
            augmented = self.augment_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        elif self.original_transform:
            augmented = self.original_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# 데이터 증강 정의
train_transform = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.OneOf([
        A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.8, alpha_coef=0.1, p=0.5),
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), p=0.5),
        A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=0.5),
    ], p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 원본 데이터를 위한 transform
original_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터셋 및 데이터로더 생성
train_dataset = AutonomousDrivingDataset(root_dir='training', original_transform=original_transform, augment_transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

# 모델 정의 및 수정
model = deeplabv3_mobilenet_v3_large(pretrained=False, num_classes=25)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Early stopping 클래스
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# 학습 실행
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 24
early_stopping = EarlyStopping(patience=5, min_delta=0.001)
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, train_loader, criterion, device)  # 실제로는 별도의 검증 데이터셋을 사용해야 합니다
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # 최상의 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_autonomous_driving_segmentation_model.pth')
        print(f"Saved best model at epoch {epoch+1}")
    
    # Early stopping 체크
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    
    # 3 에폭마다 모델 저장
    if (epoch + 1) % 3 == 0:
        torch.save(model.state_dict(), f'autonomous_driving_segmentation_model_epoch_{epoch+1}.pth')
        print(f"Saved model at epoch {epoch+1}")

# 최종 모델 저장
torch.save(model.state_dict(), 'final_autonomous_driving_segmentation_model.pth')
print("Training complete. Final model saved.")


/home/mira/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mira/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /home/mira/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:01<00:00, 11.7MB/s]
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file


Epoch 1/24, Train Loss: 0.4663, Val Loss: 0.3685
Saved best model at epoch 1


Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file


Epoch 2/24, Train Loss: 0.3349, Val Loss: 0.3555
Saved best model at epoch 2


Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
